In [ ]:
!pip install docling


In [ ]:
!pip install torch


In [ ]:
!pip install transformers


In [ ]:
!pip install requests


In [ ]:
!pip install huggingface-hub


In [ ]:
!pip install tqdm

In [ ]:
import requests
import os

pdfs = {
    "MALM": "https://www.ikea.com/us/en/assembly_instructions/malm-4-drawer-chest-white__AA-2398381-2-100.pdf",
    "BILLY": "https://www.ikea.com/us/en/assembly_instructions/billy-bookcase-white__AA-1844854-6-2.pdf",
    "BOAXEL": "https://www.ikea.com/us/en/assembly_instructions/boaxel-wall-upright-white__AA-2341341-2-100.pdf",
    "MICKE": "https://www.ikea.com/us/en/assembly_instructions/micke-desk-white__AA-476626-10-100.pdf"
}

output_dir = "data"
os.makedirs(output_dir, exist_ok=True)

for name, url in pdfs.items():
    response = requests.get(url)
    pdf_path = os.path.join(output_dir, f"{name}.pdf")

    with open(pdf_path, "wb") as f:
        f.write(response.content)

    print(f"Downloadeчd {name} to {pdf_path}")

print("Downloaded files:", os.listdir(output_dir))

In [ ]:
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode
from docling_core.transforms.chunker import HierarchicalChunker
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from pathlib import Path
import json
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor
import logging
pipeline_options = PdfPipelineOptions(do_table_structure=True)
pipeline_options.table_structure_options.mode = TableFormerMode.ACCURATE  # use more accurate TableFormer model

doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

In [ ]:
from docling_core.types.doc import DoclingDocument, PictureItem
from PIL import Image
import io

class ImageProcessor:
    def __init__(self, doc: DoclingDocument):
        self.doc = doc

    def add_image(self, image_path: str, page_num: int = 0):
        """
        Добавление изображения в DoclingDocument

        Args:
            image_path: Путь к изображению
            page_num: Номер страницы
        """
        try:
            # Создаем PictureItem
            picture = PictureItem(
                source=image_path,
                type="image",
                parent="#/body",
                children=[],
                page=page_num,
                # Дополнительные метаданные
                metadata={
                    "format": self._get_image_format(image_path),
                    "size": self._get_image_size(image_path)
                }
            )

            # Добавляем в документ
            self.doc.pictures.append(picture)

            # Обновляем структуру body
            self.doc.body.children.append(f"#/pictures/{len(self.doc.pictures)-1}")

        except Exception as e:
            logging.error(f"Error adding image: {str(e)}")
            raise

    def _get_image_format(self, image_path: str) -> str:
        """Получение формата изображения"""
        with Image.open(image_path) as img:
            return img.format

In [ ]:
from docling.document_converter import DocumentConverter
from docling_core.transforms.chunker import HierarchicalChunker
from typing import List, Dict
from pathlib import Path
import json
import os
import base64
from PIL import Image
import io

class DocumentProcessor:
    def __init__(self, chunk_size: int = 3000, chunk_overlap: int = 300):
        self.converter = DocumentConverter()
        self.chunker = HierarchicalChunker(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            split_on_headings=True
        )

    def save_image(self, image, output_dir: Path, prefix: str, page_num: int, img_num: int) -> str:
        """Сохраняет изображение и возвращает путь"""
        img_dir = output_dir / 'images'
        img_dir.mkdir(exist_ok=True)

        img_path = img_dir / f"{prefix}_page{page_num}_img{img_num}.png"
        image.save(img_path)
        return str(img_path)

    def process_document(self, document_path: str):
        try:
            doc_path = Path(document_path)
            print(f"\nОбработка документа: {doc_path.name}")

            doc_result = self.converter.convert(doc_path)

            if doc_result and doc_result.document:
                chunks = list(self.chunker.chunk(doc_result.document))

                processed_chunks = []
                images_by_page = {}

                # Извлекаем изображения из документа
                if hasattr(doc_result.document, 'pages'):
                    for page_num, page in enumerate(doc_result.document.pages, 1):
                        if hasattr(page, 'images'):
                            images_by_page[page_num] = []
                            for img_num, img in enumerate(page.images, 1):
                                try:
                                    # Конвертируем изображение в PIL Image
                                    image = Image.open(io.BytesIO(img.content))

                                    # Сохраняем изображение
                                    img_path = self.save_image(
                                        image,
                                        Path("processed_texts"),
                                        doc_path.stem,
                                        page_num,
                                        img_num
                                    )

                                    images_by_page[page_num].append({
                                        'path': img_path,
                                        'width': image.width,
                                        'height': image.height,
                                        'format': image.format
                                    })
                                except Exception as e:
                                    print(f"Ошибка при обработке изображения: {str(e)}")

                for chunk in chunks:
                    if hasattr(chunk, 'text'):
                        page_num = chunk.meta.doc_items[0].prov[0].page_no if hasattr(chunk.meta, 'doc_items') else None

                        chunk_data = {
                            'text': chunk.text,
                            'metadata': {
                                'headings': chunk.meta.headings if hasattr(chunk.meta, 'headings') else [],
                                'page_number': page_num,
                                'images': images_by_page.get(page_num, [])
                            }
                        }
                        processed_chunks.append(chunk_data)

                return processed_chunks
            else:
                print(f"Не удалось извлечь текст из {doc_path}")
                return None

        except Exception as e:
            print(f"Ошибка при обработке {document_path}: {str(e)}")
            return None

def process_all_pdfs():
    processor = DocumentProcessor()
    data_dir = Path("data")
    output_dir = Path("processed_texts")
    output_dir.mkdir(exist_ok=True)

    pdf_files = list(data_dir.glob("*.pdf"))
    print(f"Найдено PDF файлов: {len(pdf_files)}")

    for pdf_path in pdf_files:
        print(f"\nОбработка {pdf_path.name}")
        chunks = processor.process_document(str(pdf_path))


        if chunks:
            output_path = output_dir / f"{pdf_path.stem}_chunks.json"
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump({
                    'filename': pdf_path.name,
                    'chunks': chunks,
                    'total_chunks': len(chunks)
                }, f, ensure_ascii=False, indent=2)
            print(f"Сохранено чанков: {len(chunks)}")

            if chunks:
                print("\nПример первого чанка:")
                print("-" * 50)
                print(f"Текст: {chunks[0]['text'][:200]}...")
                print(f"Страница: {chunks[0]['metadata']['page_number']}")
                print(f"Заголовки: {chunks[0]['metadata']['headings']}")
                if chunks[0]['metadata']['images']:
                    print(f"Изображения на странице: {len(chunks[0]['metadata']['images'])}")

def check_results():
    output_dir = Path("processed_texts")
    if not output_dir.exists():
        print("Директория с результатами не найдена")
        return

    print("\nРезультаты обработки:")
    for json_file in output_dir.glob("*_chunks.json"):
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            print(f"\nФайл: {data['filename']}")
            print(f"Всего чанков: {data['total_chunks']}")
            if data['chunks']:
                print("Пример первого чанка:")
                print("-" * 50)
                print(f"Текст: {data['chunks'][0]['text'][:200]}")
                print(f"Изображения: {len(data['chunks'][0]['metadata']['images'])}")
                for img in data['chunks'][0]['metadata']['images']:
                    print(f"- {img['path']} ({img['width']}x{img['height']})")

if __name__ == "__main__":
    process_all_pdfs()
    print("\nПроверка результатов:")
    check_results()

Найдено PDF файлов: 5

Обработка ADILS.pdf

Обработка документа: ADILS.pdf


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Сохранено чанков: 51

Пример первого чанка:
--------------------------------------------------
Текст: For increased stability, re-tighten the screws about two weeks after assembly. Make sure they stay tight by checking them a couple of times per year....
Страница: 2
Заголовки: ['English']

Обработка BILLY.pdf

Обработка документа: BILLY.pdf
Сохранено чанков: 118

Пример первого чанка:
--------------------------------------------------
Текст: Serious or fatal crushing injuries can occur from furniture tip over. To prevent tip over this furniture must be used with the wall attachment device(s) provided....
Страница: 2
Заголовки: ['WARNING!']

Обработка MALM.pdf

Обработка документа: MALM.pdf


KeyboardInterrupt: 

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# pdf_processor/config.py
from pathlib import Path
from typing import Dict, List, Tuple

class WorkspaceConfig:
    """Конфигурация рабочего пространства"""
    DIRECTORIES = {
        'input': Path('input'),
        'processed': Path('processed'),
        'images': Path('processed/images')
    }

In [ ]:
# pdf_processor/workspace.py
from pathlib import Path
from typing import Dict
from .config import WorkspaceConfig  # Обратите внимание на точку перед импортом

def setup_workspace() -> Dict[str, Path]:
    """
    Создает структуру рабочих директорий.

    Returns:
        Dict[str, Path]: Словарь с путями к рабочим директориям
    """
    for dir_path in WorkspaceConfig.DIRECTORIES.values():
        dir_path.mkdir(parents=True, exist_ok=True)

    return WorkspaceConfig.DIRECTORIES

ImportError: attempted relative import with no known parent package

In [ ]:
# main.py
from google.colab import files
from pathlib import Path
from typing import Dict, List
import json

from workspace import setup_workspace
from file_handler import FileHandler
from document_processor import DocumentProcessor



def main():
    """Основной процесс обработки документов"""
    # Создаем рабочее пространство
    workspace = setup_workspace()

    # Импортируем файлы
    file_handler = FileHandler()
    imported_files = file_handler.import_pdfs(workspace)

    # Создаем экземпляр процессора
    processor = DocumentProcessor()

    # Обрабатываем каждый файл
    for pdf_path in imported_files:
        result = processor.process_document(str(pdf_path), workspace)

        if result:
            # Сохраняем результаты
            output_path = workspace['processed'] / f"{pdf_path.stem}_processed.json"
            file_handler.save_results(result, output_path)

            print(f"\nОбработан документ: {pdf_path.name}")
            print(f"Чанков: {len(result['chunks'])}")
            print(f"Изображений: {len(result['images'])}")

    # Скачиваем результаты
    print("\nСкачивание результатов...")
    for result_file in workspace['processed'].glob('*.json'):
        files.download(str(result_file))

if __name__ == "__main__":
    main()

ImportError: cannot import name 'setup_workspace' from 'workspace' (unknown location)

In [ ]:
# pdf_processor/file_handler.py
from google.colab import files
from pathlib import Path
from typing import List, Dict
import json

class FileHandler:
    """Обработчик файловых операций"""

    @staticmethod
    def import_pdfs(workspace: Dict[str, Path]) -> List[Path]:
        """
        Импортирует PDF файлы через интерфейс Colab.

        Args:
            workspace: Словарь с путями к рабочим директориям

        Returns:
            List[Path]: Список путей к импортированным файлам
        """
        print("Выберите PDF файлы для загрузки...")
        uploaded = files.upload()

        imported_files = []
        for filename in uploaded.keys():
            if filename.lower().endswith('.pdf'):
                dest_path = workspace['input'] / filename
                with open(dest_path, 'wb') as f:
                    f.write(uploaded[filename])
                imported_files.append(dest_path)
                print(f'Импортирован файл: {filename}')

        return imported_files

    @staticmethod
    def save_results(result: Dict, output_path: Path) -> None:
        """
        Сохраняет результаты обработки в JSON файл.

        Args:
            result: Результаты обработки документа
            output_path: Путь для сохранения результатов
        """
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False, indent=2)

In [ ]:
# pdf_processor/document_processor.py
from pathlib import Path
from typing import Dict, Optional

class DocumentProcessor:
    """Процессор документов"""

    def process_document(self, pdf_path: str, workspace: Dict[str, Path]) -> Optional[Dict]:
        """
        Обрабатывает PDF документ.

        Args:
            pdf_path: Путь к PDF файлу
            workspace: Словарь с путями к рабочим директориям

        Returns:
            Optional[Dict]: Результаты обработки или None в случае ошибки
        """
        # Здесь должна быть ваша логика обработки PDF
        # Пример заглушки:
        return {
            'chunks': ['chunk1', 'chunk2'],
            'images': ['image1.png', 'image2.png']
        }

In [ ]:
# main.py
from pathlib import Path
from typing import Dict, List
import json

from pdf_processor.workspace import setup_workspace
from pdf_processor.file_handler import FileHandler
from pdf_processor.document_processor import DocumentProcessor

def main():
    """Основной процесс обработки документов"""
    # Создаем рабочее пространство
    workspace = setup_workspace()

    # Импортируем файлы
    file_handler = FileHandler()
    imported_files = file_handler.import_pdfs(workspace)

    # Создаем экземпляр процессора
    processor = DocumentProcessor()

    # Обрабатываем каждый файл
    for pdf_path in imported_files:
        result = processor.process_document(str(pdf_path), workspace)

        if result:
            # Сохраняем результаты
            output_path = workspace['processed'] / f"{pdf_path.stem}_processed.json"
            file_handler.save_results(result, output_path)

            print(f"\nОбработан документ: {pdf_path.name}")
            print(f"Чанков: {len(result['chunks'])}")
            print(f"Изображений: {len(result['images'])}")

    # Скачиваем результаты
    print("\nСкачивание результатов...")
    for result_file in workspace['processed'].glob('*.json'):
        files.download(str(result_file))

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'pdf_processor'

In [ ]:
# main.py
from google.colab import files
from pathlib import Path
from typing import Dict, List
import json

def main():
    """Основной процесс обработки документов"""
    # Создаем рабочее пространство
    workspace = setup_workspace()

    # Импортируем файлы
    file_handler = FileHandler()
    imported_files = file_handler.import_pdfs(workspace)

    # Создаем экземпляр процессора
    processor = DocumentProcessor()

    # Обрабатываем каждый файл
    for pdf_path in imported_files:
        result = processor.process_document(str(pdf_path), workspace)

        if result:
            # Сохраняем результаты
            output_path = workspace['processed'] / f"{pdf_path.stem}_processed.json"
            file_handler.save_results(result, output_path)

            print(f"\nОбработан документ: {pdf_path.name}")
            print(f"Чанков: {len(result['chunks'])}")
            print(f"Изображений: {len(result['images'])}")

    # Скачиваем результаты
    print("\nСкачивание результатов...")
    for result_file in workspace['processed'].glob('*.json'):
        files.download(str(result_file))

if __name__ == "__main__":
    main()

Выберите PDF файлы для загрузки...


Saving Godovoi_-otchet-PAO-GMK-Norilskii_-nikel-za-2023-god.pdf to Godovoi_-otchet-PAO-GMK-Norilskii_-nikel-za-2023-god (4).pdf
Импортирован файл: Godovoi_-otchet-PAO-GMK-Norilskii_-nikel-za-2023-god (4).pdf


TypeError: DocumentProcessor.process_document() takes 2 positional arguments but 3 were given